This notebook demonstrates how to:
1. Load sample tracking data from the NFL Big Data Bowl dataset  
2. Run the trained deep learning model (`SeqInterModel`)  
3. Visualize predicted player trajectories vs. ground truth


In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.model import SeqInterModel
from src.dataset import build_seq_samples_for_play

In [ ]:
DATA_DIR = "/data/train"

# Example files (adjust names if different)
f_in = os.path.join(DATA_DIR, "week01_input.csv")
f_out = os.path.join(DATA_DIR, "week01_output.csv")

df_in = pd.read_csv(f_in)
df_out = pd.read_csv(f_out)

print(f"Input shape: {df_in.shape} | Output shape: {df_out.shape}")
print(df_in.head(3))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "../models/best_model.pth"

model = SeqInterModel()
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device).eval()

print("Model loaded from:", model_path)

In [ ]:
# Choose one play from the dataset
gid, pid = df_in["game_id"].iloc[0], df_in["play_id"].iloc[0]
print(f"Selected play: game_id={gid}, play_id={pid}")

sample = build_seq_samples_for_play(gid, pid, df_in, df_out)
if sample is None:
    raise ValueError("Play not found or invalid data!")

T = sample["T"]
past = sample["past"].to(device)
nbr_idx = sample["nbr_idx"].to(device)
ball_ctx = sample["ball_land"].to(device)
x0, y0 = past[:, -1, 0], past[:, -1, 1]
dt = sample["dt_eff"]

with torch.no_grad():
    pred = model(past, nbr_idx, ball_ctx, T, x0, y0, v_max=10.14, dt=dt)
pred = pred.cpu().numpy()
print("Prediction shape:", pred.shape)


In [ ]:
# Choose one player to visualize
i = 0  # index in the player list
gt = sample["target"][i].numpy()
pred_i = pred[i]

plt.figure(figsize=(8, 6))
plt.plot(gt[:, 0], gt[:, 1], 'o-', label="Ground truth", alpha=0.6)
plt.plot(pred_i[:, 0], pred_i[:, 1], 'o--', label="Prediction", alpha=0.8)
plt.xlabel("X position (yards)")
plt.ylabel("Y position (yards)")
plt.legend()
plt.title(f"Player Trajectory Prediction — game_id={gid}, play_id={pid}")
plt.grid(True)
plt.show()

In [ ]:
# plot all players
plt.figure(figsize=(10, 7))
for i in range(pred.shape[0]):
    gt = sample["target"][i].numpy()
    pred_i = pred[i]
    plt.plot(gt[:, 0], gt[:, 1], '-', color='gray', alpha=0.3)
    plt.plot(pred_i[:, 0], pred_i[:, 1], '--', color='orange', alpha=0.6)
plt.title(f"Predicted vs. True Player Trajectories\nGame {gid} | Play {pid}")
plt.xlabel("X (yards)")
plt.ylabel("Y (yards)")
plt.grid(True)
plt.show()

In [ ]:
# save predictions
nfl_ids = sample["nfl_ids"]
frames = range(T)

df_pred = pd.DataFrame(
    [(gid, pid, int(nfl_ids[i]), int(f), pred[i, f, 0], pred[i, f, 1])
     for i in range(len(nfl_ids)) for f in frames],
    columns=["game_id", "play_id", "nfl_id", "frame_id", "x_pred", "y_pred"]
)
df_pred.to_csv("../predictions_sample.csv", index=False)
print("💾 Saved predictions to ../predictions_sample.csv")